## 迭代法

对于大型稀疏矩阵求解线性方程组, 倘若使用高斯消元法会使矩阵变得稠密, 可能会消耗大量内存. 迭代法是可以适应稀疏矩阵的一种方法.

### Jacobi 迭代法

以 $3\times 3$ 的 $Ax = b$ 问题为例, 假设现在已解出近似解 $x = [x_1,\dotsc,x_n]^T$, 令
$$x_1' = (b_1 - a_{12}x_2 - a_{13}x_3)/ a_{11}\\
  x_2' = (b_2 - a_{21}x_1 - a_{23}x_3)/ a_{22}\\
  x_3' = (b_3 - a_{31}x_1 - a_{32}x_2)/ a_{33}$$
算作一次迭代 . 迭代规则很好理解, 即 $x_k' = (b_k - \sum_{i\neq k}a_{ki}x_i) / a_{ki}$.

若记 $$A = \left[\begin{matrix}a_{11} & a_{12} & a_{13} & \dotsc & a_{1n}\\
a_{21} & a_{22} & a_{23} &\dotsc &a_{2n}\\
a_{31} & a_{32} & a_{33} &\dotsc &a_{3n}\\
\vdots & \vdots &\vdots &\ddots &\vdots \\
a_{n1} & a_{n2} & a_{n3} &\dotsc &a_{nn}
\end{matrix}\right]$$

令对角元构成矩阵 $D= {\rm diag}[a_{11},a_{22},\dotsc,a_{nn}]$, 以及 $L,U$ 为严格下三角与严格上三角:
$$L = \left[\begin{matrix}  &   &  &   & \\
a_{21} &   &  &  & \\
a_{31} & a_{32} &  & & \\
\vdots & \vdots &\ddots & & \\
a_{n1} & a_{n2} & a_{n3} &\dotsc &
\end{matrix}\right]\quad 
U= \left[\begin{matrix}\  & a_{12}  &a_{12}  &\dotsc   &a_{1n} \\
\ &   & a_{23}&\dotsc  &a_{2n} \\
\ &   &  & \ddots & a_{3n} \\
\ &   &  & &\vdots \\
\ &  &  &  &
\end{matrix}\right].$$

那么 $A = D + L + U$. 则该迭代可以写成向量形式

$$ x' = D^{-1}(b - (L+U)x).$$

这是线性递推, 考察其不动点 $x_* = D^{-1}(b - (L+U)x_*)$ 解得 $Ax_* =b$, 即不动点为精确解. 同时

$$ x' - x_* = -D^{-1}(L+U)(x - x_*).$$

或 $$x^{(n)} - x_* = (-D^{-1}(L+U))^n(x^{(0)}-x_*).$$

因此若上式对于任意初始值, $n\rightarrow \infty$ 迭代收敛当且仅当 $D^{-1}(L+U)$ 的谱半径小于 $1$. 且极限为精确解 $x_*$. 

In [2]:
import numpy as np 
def JacobiIteration(A,b,iter = -1):
    eps = np.linalg.norm(A) * 2e-15
    n = A.shape[0]
    x = np.random.randn(n)
    d = np.zeros(n,dtype=A.dtype)
    if iter < 0: iter = 200 * n
    # 扔掉 A 的对角元, 变成 -(L+U)
    for i in range(n):
        d[i] = A[i,i]
        A[i,i] = 0
    
    for _ in range(iter):
        if np.linalg.norm(A @ x + d * x - b) <= eps:
            break
        x = (b - A @ x) / d

    for i in range(n): # 还原
        A[i,i] = d[i] 

    return x

In [3]:
# 构造一个 D^{-1}(L+U) 谱半径必然小于 1 的矩阵
n = 500
np.random.seed(3)
A = np.random.randn(n*n).reshape((n,n))
for i in range(n):
    A[i,i] = 0
s = max(np.abs(np.linalg.eigvals(A)))
for i in range(n):
    A[i,i] = s + abs(np.random.randn())

realsol = np.random.randn(n)
b = A @ realsol
x = JacobiIteration(A,b)
print('Error    =',np.linalg.norm(x - realsol))
print('Residual =',np.linalg.norm(A @ x - b))

Error    = 5.3466819503992217e-14
Residual = 1.4001496497771627e-12


### Gauss-Seidel 迭代法

仍然以 $3\times 3$ 为例, 现在
$$x_1' = (b_1 - a_{12}x_2 - a_{13}x_3)/ a_{11}\\
  x_2' = (b_2 - a_{21}x_1' - a_{23}x_3)/ a_{22}\\
  x_3' = (b_3 - a_{31}x_1' - a_{32}x_2')/ a_{33}$$
即更新$x_k$ 的时候利用已经更新过的 $x_1,\dotsc,x_{k-1}$. 这被称为 Gauss-Seidel 迭代法. 采用方才的记号,
$$x' = D^{-1}(b - Lx' - Ux).$$

亦即

$$x' = (D+L)^{-1}(b - Ux).$$

保证收敛当且仅当 $(D+L)^{-1}U$ 的谱半径 $ < 1$. 注意, Jacobi 迭代法收敛和 Gauss-Seidel 迭代法收敛条件之间不存在充分或必要关系 (可能一个收敛但另一个不收敛).

可以证明, 若 $A$ 对称正定, 则 Gauss-Seidel 方法必收敛. [1] p. 112.

In [4]:
from numba import jit
@jit(nopython = True)
def GaussSeidel(A,b,iter = -1):
    eps = np.linalg.norm(A) * 2e-15
    n = A.shape[0]
    x = np.random.randn(n)
    if iter < 0: iter = 100 * n
    for _ in range(iter):
        if np.linalg.norm(A @ x - b) <= eps:
            break
        for k in range(n):
            #x[k] =  (b[k] - np.dot(A[k],x) + A[k,k]*x[k]) / A[k,k]
            x[k] += (b[k] - np.dot(A[k],x)) / A[k,k]
    return x

n = 500
np.random.seed(1)
A = np.random.randn(n*n).reshape((n,n))
A = A.T @ A + np.eye(n) * np.random.random()

realsol = np.random.randn(n)
b = A @ realsol
x = GaussSeidel(A,b)
print('Error    =',np.linalg.norm(x - realsol))
print('Residual =',np.linalg.norm(A @ x - b))

Error    = 3.5226807513908706e-11
Residual = 3.157780465137217e-11


## References

1. 徐树方,高立,张平文, 数值线性代数, The Peking University Press, 2nd ed., 2013.